In [73]:
import requests
import pandas as pd
flag = [1,2,3,4,5,6,7,8,9,10]
urllist = []
for i in flag:
    url = f"https://itunes.apple.com/tw/rss/customerreviews/page={i}/id=758103884/sortby=mostrecent/json"
    urllist.append(url)   # Uber 1058959277  # Foodpanda 758103884
rating = [] 
title = [] 
content = [] 
time = []

for url in urllist:
    res = requests.get(url)
    data = res.json()['feed']['entry']
    for i in range(len(data)):
        rating.append(data[i]['im:rating']['label'])
        title.append(data[i]['title']['label'])
        content.append(data[i]['content']['label'])
        time.append(data[i]['updated']['label'])

data = {
        '時間':time,
       '標題':title,
       '內容':content,
        '評分':rating,
        }
df = pd.DataFrame(data)
df['內容'] = df['內容'].replace(r"\n",' ',regex=True)
df['內容'] = df['內容'].replace(r"\n",' ',regex=True)

In [74]:
#賦予每則留言編號
ID = []
for index,row in df.iterrows():
    id = index+1 
    ID.append(id)
df['ID'] = ID

In [75]:
#時間調整
from datetime import datetime, timedelta, timezone
df['Datetime'] = pd.to_datetime(df['時間'], utc=False)
df['Date'] = df['Datetime'].dt.tz_convert('Asia/Shanghai')
df.drop(columns=['Datetime'],inplace=True)
df.drop(columns=['時間'],inplace=True)
df['時間'] = df['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df.drop(columns=['Date'],inplace=True)

In [76]:
#輸出 comments
df.to_csv(r'comments.csv',encoding='utf_8_sig')

In [77]:
import jieba 
import jieba.analyse
jieba.set_dictionary('Tools\dict.txt')  # 繁體字典
with open('Tools\stops.txt', 'r', encoding='utf8') as f:  # 中文的停用字
    stops = f.read().split('\n') 

#新增停用詞
s = ['沒','都','沒有','很','不','不要','請','好','嗎','因為','更','後','月','表示','年','以來']

for w in s:
    stops.append(w)
    
#自訂義辭典
jieba.load_userdict(r'Tools\userDict.txt')    
    
#斷詞並刪除停用詞
def cut(mytext):
    terms = [t for t in jieba.cut(mytext) if t not in stops]
    return " ".join(terms)

Building prefix dict from C:\Users\19749\Desktop\Python\Text Mining\Tools\dict.txt ...
Loading model from cache C:\Users\19749\AppData\Local\Temp\jieba.u55e26c21e268e022ceb9200c7c630427.cache
Loading model cost 1.208 seconds.
Prefix dict has been built successfully.


In [78]:
#針對每則留言做關鍵字
df_tag = pd.DataFrame(columns=['ID','keyword','weight'])
for index, row in df.iterrows():
    id, context = row['ID'], row['內容']
    tags = jieba.analyse.extract_tags(context, topK=5, withWeight=True, allowPOS=())
    df_ = pd.DataFrame(tags, columns=['keyword','weight'])
    df_['ID'] = id
    df_tag = pd.concat([df_tag, df_])

In [79]:
#輸出keyword
df_tag.to_csv(r'df_tag.csv',encoding='utf_8_sig')